In [89]:
import pandas as pd
import numpy as np

In [90]:
df=pd.read_csv('/kaggle/working/train_prep_new.csv')

In [146]:
features=df.drop('Loan Sanction Amount (USD)',axis=1)
label=df['Loan Sanction Amount (USD)']
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler
xtrain,xtest,ytrain,ytest=train_test_split(features,label,random_state=42,test_size=0.1)

In [147]:
sc=RobustScaler()
xtrain=sc.fit_transform(xtrain)
xtest=sc.transform(xtest)
sc2=RobustScaler()
ytrain=sc2.fit_transform(ytrain.values.reshape(-1,1))
ytest=sc2.transform(ytest.values.reshape(-1,1))

In [148]:
ytrain=ytrain.reshape(-1,)
ytrain

array([ 1.32531892, -0.48853903,  0.97676359, ...,  0.38226877,
        0.09999186,  0.26460318])

In [149]:
ytest=ytest.reshape(-1,)

In [150]:
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
param={'n_estimators':np.arange(70,150,10),
       'max_features':['log2','sqrt','auto'],'max_depth':[2,3,4,5]}
rcv=RandomizedSearchCV(estimator=GradientBoostingRegressor(),param_distributions=param,scoring='r2')
rcv.fit(xtrain,ytrain)

RandomizedSearchCV(estimator=GradientBoostingRegressor(),
                   param_distributions={'max_depth': [2, 3, 4, 5],
                                        'max_features': ['log2', 'sqrt',
                                                         'auto'],
                                        'n_estimators': array([ 70,  80,  90, 100, 110, 120, 130, 140])},
                   scoring='r2')

In [151]:
rcv.best_params_

{'n_estimators': 80, 'max_features': 'auto', 'max_depth': 5}

In [152]:
rf=GradientBoostingRegressor(n_estimators=80,max_features='auto',max_depth=5)
rf.fit(xtrain,ytrain)

GradientBoostingRegressor(max_depth=5, max_features='auto', n_estimators=80)

In [153]:
pred=rf.predict(xtest)

In [154]:
from sklearn.metrics import r2_score
print(r2_score(ytest,pred))

0.7830591607796329


In [161]:
train=df.drop('Loan Sanction Amount (USD)',axis=1)
sc2=RobustScaler()
label=df['Loan Sanction Amount (USD)']
label=sc2.fit_transform(label.values.reshape(-1,1))
label=label.reshape(-1,)
sc=RobustScaler()
train=sc.fit_transform(train)
rf=GradientBoostingRegressor(n_estimators=80,max_features='auto',max_depth=5) # yielded r2 of 81.10054 --> RandomForest
rf.fit(train,label) # 81.13340 GBR(est=80,features=auto,depth=5)

GradientBoostingRegressor(max_depth=5, max_features='auto', n_estimators=80)

In [162]:
# test---
test=pd.read_csv('/kaggle/working/test_prep_new.csv')
test=test[df.columns[:-1]]

In [163]:
test=sc.transform(test)

In [164]:
pred=rf.predict(test)

In [165]:
pred

array([ 0.71182325,  0.40425195, -0.42229951, ..., -0.51387017,
        0.48804612,  0.45347161])

In [171]:
pred=sc2.inverse_transform(pred.reshape(-1,1))

In [172]:
submit=pd.read_csv('/kaggle/input/cipla-input/test.csv')

submit=submit['Customer ID']
submit=pd.DataFrame(submit)

submit['Loan Sanction Amount (USD)']=pred

submit

,Customer ID,Loan Sanction Amount (USD)
0,C-26247,88570.822351
1,C-35067,65883.277136
2,C-34590,4913.921133
3,C-16668,46903.849519
4,C-12196,65467.695183
...,...,...
19995,C-9076,93429.511172
19996,C-17587,28950.316350
19997,C-46479,-1840.653701
19998,C-3099,72064.230612


In [175]:
submit['Loan Sanction Amount (USD)'][submit['Loan Sanction Amount (USD)']<0]=0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [178]:
submit.to_csv('submit2.csv',index=False)